#### Section 1.4.1 Voltage Dividers
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

#### Imports

In [11]:
from pathlib import Path
import py4spice as spi

#### Path to Ngspice and the project

In [12]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/sec_1_04_02_lin_reg")

#### Prepare simulation results directory

In [13]:
# Create sim_results directory if doesn't exist
RESULTS_PATH = PROJ_PATH / "sim_results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

#### Define netlists

In [14]:
# Create netlists directory and files if they don't exist
NETLISTS_PATH = PROJ_PATH / "netlists"
NETLISTS_PATH.mkdir(parents=True, exist_ok=True)  # Create the directory

# Create title.cir with content only if it doesn't exist
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
if not TITLE_FILENAME.exists():
    with open(TITLE_FILENAME, "w") as f:
        f.write("* First line in netlist must be a comment")
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)

# These netlist objects will be combined and a top netlist will be created later
dut = spi.Netlist(NETLISTS_PATH / "dut.cir")
load = spi.Netlist(NETLISTS_PATH / "load.cir")
stimulus = spi.Netlist(NETLISTS_PATH / "stimulus.cir")
supplies = spi.Netlist(NETLISTS_PATH / "supplies.cir")
models = spi.Netlist(NETLISTS_PATH / "models.cir")

# two files we'll create later
TOP_FILENAME = NETLISTS_PATH / "top.cir"
CONTROL_FILENAME = NETLISTS_PATH / "control.cir"

Convert easyeda_dut.cir to dut.cir

In [15]:
easyeda = spi.Netlist(NETLISTS_PATH / "easyeda_dut.cir")
easyeda.del_line_starts_with("*") # remove first line (comment)
easyeda.del_line_starts_with("veasyeda") # remove fake voltage source
easyeda.del_line_starts_with(".tran 1m 3.1415") # remove fake transient analysis
easyeda.del_line_starts_with(".inc standard.bjt") # remove model include
dut = easyeda
dut.write_to_file(NETLISTS_PATH / "dut.cir") # this is now the dut (device under test)

#### Create vectors

In [16]:
VEC_ALL = spi.Vectors("all")
VEC_ALL_EXPANDED = spi.Vectors(
    "q1_base div e1#branch in out out_meas vee vee#branch vin#branch vmeas#branch vref vref1#branch"
)
VEC_IN_OUT = spi.Vectors("in out")
VEC_INTEREST = spi.Vectors("in out div q1_base")
VEC_POWER_CALC = spi.Vectors("in vin#branch out vmeas#branch")
VEC_ETA = spi.Vectors("eta")

#### Define analyses

In [7]:
list_of_analyses: list[spi.Analyses] = [] # create empty list. Next sections define

# 1st analysis: operating point
# for this simulation, we'll have only one analysis
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(op1)

# 2nd analysis
dc_cmd = "dc vin 6 17 0.1"
dc1 = spi.Analyses("dc1", "dc", dc_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses.append(dc1)

#### Create control section

In [8]:
my_control = spi.Control()  # create 'my_control' object
my_control.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses:  # statements for all analyses
    my_control.insert_lines(analysis.lines_for_cntl())
spi.print_section("Control File", my_control)  # print out contents
my_control.content_to_file(CONTROL_FILENAME)  # creat the actual file
control = spi.Netlist(NETLISTS_PATH / CONTROL_FILENAME) # create netlist object


--- Control File ---
.control
* Timestamp: Sun Feb 11 02:15:45 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
--------------------



#### Simulate

In [9]:
# Combine the netlists and write out into one top netlist, ready to simulate
top = title + dut + load + supplies + stimulus + models + control

top.write_to_file(TOP_FILENAME)
spi.print_section("top netlist", top)

# prepare simulate object, print out command, and simulate
sim1 = spi.Simulate(NGSPICE_EXE, TOP_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run() # run the Ngspice simulation


--- top netlist ---
* two voltage dividers
r1 in out1 140
r2 in out2 132
rload1 out1 out1_meas 100
rload2 out2 out2_meas 50
vmeas1 out1_meas 0 dc 0
vmeas2 out2_meas 0 dc 0
vee vee 0 dc 0
vin in 0 dc 12
.control
* timestamp: sun feb 11 02:15:45 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/sim_results/op1.txt
quit
.endc
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_book_sim/circuits/sec_1_04_01_dividers/netlists/top.cir
-----------------------


Note: No compatibility mode selected!


Circuit: * two voltage dividers

	* two voltage dividers

     2 : .global gnd
     3 : r1 in out1 140
     4 : r2 in out2 132
     5 : rload1 out1 out1_meas 100
     6 : rload2 out2 out2_meas 50
     7 : vmeas1 out1_meas 0 dc 0
     8 : vmeas2 out2_meas 0 dc 0
     9 : vee vee 0 dc 0
    10 : vin in 0 dc 12
    11 : .contr

#### Gather simulation results and transform into more usable format

In [10]:
# Path and name of result file
RESULTS_FILENAME = RESULTS_PATH / f"{list_of_analyses[0].name}.txt"

op1_signals = spi.Signals.from_spice_table(RESULTS_FILENAME) # create signals object
op1_values = op1_signals.signals_table(VEC_IN_OUT.list_out())
spi.print_section("Operating Point Results", op1_values)


--- Operating Point Results ---
Signal values for x = 0
out2:  3.2967
out1:  5
  in:  12
-------------------------------



#### Calculations and printout

In [11]:
vin = op1_signals.signal_value(str(VEC_VIN))
iin = -1 * op1_signals.signal_value(str(VEC_IIN))
vout1 = op1_signals.signal_value(str(VEC_VOUT1))
vout2 = op1_signals.signal_value(str(VEC_VOUT2))
iout1 = op1_signals.signal_value(str(VEC_IOUT1))
iout2 = op1_signals.signal_value(str(VEC_IOUT2))
pr1 = (vin - vout1) * iout1
pr2 = (vin - vout2) * iout2
pout1 = vout1 * iout1
pout2 = vout2 * iout2
pout = pout1 + pout2
pin = vin * iin
eta = pout / pin

print(f"p1 = {pr1:.4g}W")
print(f"p2 = {pr2:.4g}W")
print(f"pout1 = {pout1:.4g}W")
print(f"pout2 = {pout2:.4g}W")
print(f"pout = {pout:.4g}W")
print(f"pin = {pin:.4g}W")
print(f"eta = {eta*100:.4g}%")

p1 = 0.35W
p2 = 0.5738W
pout1 = 0.25W
pout2 = 0.2174W
pout = 0.4674W
pin = 1.391W
eta = 33.59%
